In [70]:
from __future__ import print_function

import os
import sys
import numpy as np

import pickle

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import layers, Sequential
from keras.utils import to_categorical
from keras.initializers import Constant

import pandas as pd

In [135]:
BASE_DIR = ''
GLOVE_DIR = BASE_DIR
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS = 1000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [73]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [136]:
dataset = pd.read_csv('aclImdb/train.csv')
texts = dataset['text'].values
labels = dataset['target'].values
labels_index = {
    1: 1,
    0: 0,
}

In [137]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19390 unique tokens.


In [138]:
with open('tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)

In [145]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (1000, 200)
Shape of label tensor: (1000, 2)


In [146]:
X_train, X_test, y_train, y_test \
    = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val \
    = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Scikit`s SVC

In [141]:
embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [118]:
X = []
for row in X_train:
    arr = []
    for foo in row:
        arr.extend(embedding_matrix[foo])
    X.append(arr)
X_train = np.array(X)

In [119]:
X = []
for row in X_test:
    arr = []
    for foo in row:
        arr.extend(embedding_matrix[foo])
    X.append(arr)
X_test = np.array(X)

In [120]:
X_test.shape

(200, 100000)

In [121]:
X_train.shape

(640, 100000)

In [122]:
model = SVC()

In [123]:
model.fit(X_train, y_train)

/Users/vlsnk/anaconda3/envs/fucked-up-data-science/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [124]:
y_pred = model.predict(X_test)

In [125]:
accuracy_score(y_test, y_pred)

0.495

## Tensorflow

In [117]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros(shape=(num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [88]:
embedding_layer = layers.Embedding(
    num_words,
    EMBEDDING_DIM,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [111]:
model = Sequential([
    layers.Embedding(
        num_words,
        EMBEDDING_DIM,
        input_shape=(MAX_SEQUENCE_LENGTH,),
        embeddings_initializer=Constant(embedding_matrix),
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False
    ),
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(5),
    layers.Conv1D(128, 5, activation='relu'),
    layers.MaxPooling1D(5),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels_index), activation='softmax')
])

In [150]:
model = Sequential([
    layers.Embedding(
        num_words,
        EMBEDDING_DIM,
        input_shape=(MAX_SEQUENCE_LENGTH,),
        embeddings_initializer=Constant(embedding_matrix),
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False
    ),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels_index), activation='softmax')
])

In [151]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [152]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 640 samples, validate on 160 samples
Epoch 1/10
640/640 [==============================] - 0s 716us/step - loss: 0.8623 - acc: 0.4906 - val_loss: 0.7748 - val_acc: 0.4625
Epoch 2/10
640/640 [==============================] - 0s 46us/step - loss: 0.7434 - acc: 0.4812 - val_loss: 0.6953 - val_acc: 0.5375
Epoch 3/10
640/640 [==============================] - 0s 48us/step - loss: 0.7012 - acc: 0.5328 - val_loss: 0.6928 - val_acc: 0.5688
Epoch 4/10
640/640 [==============================] - 0s 39us/step - loss: 0.7548 - acc: 0.4938 - val_loss: 0.7168 - val_acc: 0.4500
Epoch 5/10
640/640 [==============================] - 0s 37us/step - loss: 0.7436 - acc: 0.4500 - val_loss: 0.6859 - val_acc: 0.5688
Epoch 6/10
640/640 [==============================] - 0s 37us/step - loss: 0.6942 - acc: 0.5297 - val_loss: 0.7190 - val_acc: 0.4500
Epoch 7/10
640/640 [==============================] - 0s 37us/step - loss: 0.7204 - acc: 0.4922 - val_loss: 0.6827 - val_acc: 0.5562
Epoch 8/10
640/640 [==

In [153]:
loss, acc = model.evaluate(X_test, y_test)
print('Accuracy is', acc)

200/200 [==============================] - 0s 78us/step
Accuracy is 0.4749999940395355
